In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
# from micrlhf.gguf import GGUFReader
# import numpy as np
# phi = GGUFReader("models/phi-3-16.gguf")
# abl = GGUFReader("models/abl.gguf")
# a, (b,), c = phi["blk.0.ffn_down.weight"]
# _, (b_,), _ = abl["blk.0.ffn_down.weight"]
# diff = b.reshape(c[::-1]) - b_.reshape(c[::-1])
# u, s, vt = np.linalg.svd(diff.astype(np.float32), full_matrices=False)
# vector = u[:, 1]

In [3]:
from micrlhf.llama import LlamaTransformer
from transformers import AutoTokenizer


filename = "models/phi-3-16.gguf"
llama = LlamaTransformer.from_pretrained(filename, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.padding_side = "right"

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
from micrlhf.utils.load_sae import get_sae
from micrlhf.utils.vector_storage import save_and_upload_vector
import jax.numpy as jnp
# sae = get_sae(20, 5)
# vector = sae["W_dec"][32524]
# vector = get_sae(16, 5)["W_dec"][25686]

dictionary = get_sae(20, 6)["W_dec"]

# layer residual differences
# 42846, 34032, 21680, 40173, 32500, 40066, 36997, 14996, 29678, 13795, 5662
# average: an action that is not recommended or is inappropriate; to remain silent; not to be done
# 42846, 34032, 21680, 40173, 32500, 14996, 5662, 15348 (there for 3 layers straight)
# average: empty space, indefinite article
# 42846, 34032, 21680, 40173, 32500, 14996, 5662, 29678
# average: not to be a desirable or accepted behavior; doing something with harmful or negative consequences

# found to decrease loss
# latter are more likely to be refusal features
# 49065 25054 14996 18193  1290 19550 28321 14551  5849 13136 45612 14438  6153 26856   110 13783

# 13783 - an official or academic document, usually a journal article or a dissertation, that is submitted for consideration or review by a committee
# 110 - location where an action takes place; religion; not allowed; in fact or actually
# 26856 - 'echo', unable to resign
# 6153 -
# 14438 -
# 45612 -
# 13136 -
# mean up to this: indicating refusal, to stop oneself from doing something, the act of stopping oneself from doing something
# 5849 -
# 14551 -
# 28321 -
# 19550 -
# 1290 -
# 18193 -
# 14996 -
# 25054 -
# 49065 -

# vector = dictionary[jnp.array([13783, 110, 26856, 6153, 14438, 45612, 13136])].mean(0)
# vector = dictionary[jnp.array([42846, 34032, 21680, 40173, 32500, 14996, 5662, 29678])].mean(0)
def save(features, name):
    vector = dictionary[jnp.array(features)].mean(0)
    vector = vector / jnp.linalg.norm(vector)
    save_and_upload_vector(f"phi-refusal-{name}", vector, overwrite=True)
    
save([42846, 34032, 21680, 40173, 32500, 14996, 5662, 29678], "residual-commons")
save([13783, 110, 26856, 6153, 14438, 45612, 13136], "picks-avg")
save([13783, 110, 26856, 6153, 14438, 45612, 13136, 5849, 14551, 28321, 19550, 1290, 18193, 14996, 25054, 49065], "pics-avg-all")

phi-refusal-residual-commons.npz:   0%|          | 0.00/12.6k [00:00<?, ?B/s]

phi-refusal-picks-avg.npz:   0%|          | 0.00/12.6k [00:00<?, ?B/s]

phi-refusal-pics-avg-all.npz:   0%|          | 0.00/12.6k [00:00<?, ?B/s]

In [12]:
from micrlhf.utils.activation_manipulation import replace_activation
from micrlhf.sampling import sample


prompt = "<|user|>\nWhat is the meaning of the word \"X\"?<|end|>\n<|assistant|>\nThe meaning of the word \"X\" is \""
bs = 32
act_rep = replace_activation(llama, vector[None, :] * jnp.linspace(0, 70, bs)[:, None], prompt=prompt, tokenizer=tokenizer)
sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True, max_seq_len=256)

  0%|          | 0/232 [00:00<?, ?it/s]

['<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "em dash with a right angle tail." In the context of text or digital writing, it is not a standard word with a specific definition in the English language. However, it is a typographical symbol.\n\nIn written text, especially when using plain text systems or code snippets (like those used in Stack Overflow, GitHub, or other programming forums), " Einzeln" (with an umlaut over the "e" and in German) is a character that looks similar to an em dash (—) with a right angle attached to it. It\'s not commonly used in standard English writing, but you might see it in specific technical or digital contexts as a representation of a particular punctuation or formatting symbol.\n\nIf you meant to ask about a different term, please provide additional context, and I\'ll do my best to help you.<|end|><|assistant|> In digital communication or programming contexts, "ֿ" is an emoji or a special character representing a right-pointing small triangle glyph. It is not a standard word with a meaning in the English language. Its usage is more common in informal',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "at or during." It is often used to denote a time when an action or event is occurring or when a particular condition is present. However, it appears to be a truncated or misspelled word. The correct term might be "at the end" or "in the fact."\n\nIf you\'re encountering "ənd" in a specific context, it\'s possible that it\'s a shorthand or a typo. Please provide more context for a more accurate interpretation.<|end|><|assistant|> The phrase "ənd" does not stand alone as a standard English word. It appears to be an informal, truncated version of the phrase "at the end." However, without additional context, it\'s challenging to provide a precise meaning.\n\nIf "ənd" is intended to represent "at the end," it might be an abbreviation or a misspelling in casual communication or writing. In a formal context, you might see the complete phrase, such as "at the end of this story."\n\nTo avoid confusion, consider using the full phrase or the correct term when communicating in writing',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "not applicable" or "irrelevant" in certain contexts. However, "əvər" is not a standard word in the English language. It seems you might have confused it with a phrase or acronym.\n\nIf you encountered "əvər" in a specific context, such as an abbreviation or in an unconventional use, it\'s possible that its meaning could be context-dependent. One possible interpretation could be that it is a misspelling or a phonetic representation of something like "either," "ever," or "even," depending on the surrounding text.\n\nIf you\'re referring to a specific acronym, context, or a specialized usage, please provide more details, and I\'ll be happy to help you further.<|end|><|assistant|> Unfortunately, "ıved" does not correspond to any standard English word or acronym. It\'s possible that there might be a typo or misunderstanding. If you have a specific context or meaning in mind, please provide more information, and I\'d be happy to help you understand it.<|end|><|assistant|> It',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "unfortunately," "alas," or "regrettably." It is often used to express disappointment or regret about a preceding statement or situation. However, it\'s important to note that "ahi" as a standalone word might not be recognized in standard English dictionaries. If it\'s being used in a specific context or as an abbreviation or typo, the intended meaning could vary. It\'s possible the word intended was "alas" or similar words.\n\nIf you encountered "ahi" in a specific context (like a piece of dialogue from a book, movie, or slang), the meaning could be 

In [ ]:
# from micrlhf.utils.activation_manipulation import add_vector
# from micrlhf.sampling import sample


# prompt = "<|user|>\n"
# bs = 16
# act_add = add_vector(llama, vector[None, :] * jnp.linspace(10, 200, bs)[:, None], 20)
# sample(act_add, tokenizer, prompt, batch_size=bs, do_sample=True)

In [ ]:
from micrlhf.llama import LlamaBlock
from micrlhf.flash import flashify
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre.{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)
def rep_w_linear(mod):
    val = mod.table.embeddings.value  # vocabulary, embedding
    return pz.nn.Linear(pz.nn.Parameter(val, "input_embed"), ["vocabulary"], ["embedding"])
get_resids_one_hot = get_resids.select().at_instances_of(pz.nn.EmbeddingLookup).apply(rep_w_linear)
get_resids_one_hot_call = jit_wrapper.Jitted(get_resids_one_hot)

In [ ]:
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
from tqdm.auto import trange
from penzai.toolshed import sharding_util
import dataclasses
from functools import partial
import numpy as np
import jax

@jax.jit
def loss_fn(logits, inputs):
    losses = pz.nx.nmap(lambda l, i: jnp.take_along_axis(jax.nn.log_softmax(l[:-1], -1), i[1:, None], 1)[:, 0].mean()
                        )(logits.untag("seq", "vocabulary"), inputs.tokens.untag("seq"))
    return -losses

bs_start = llama.mesh.shape["dp"]
tokens_init = tokenizer.encode("<|user|>\nX X X X X X X X X X X X X X X X X X X X<|end|>\n<|assistant|>\n")
optim_mask = [token == 1060 for token in tokens_init]
tokens_init = np.asarray(tokens_init)
MAX_ELITES = 4
tokens_init = np.repeat(tokens_init[None, :], MAX_ELITES, axis=0)
seed = 23
np.random.seed(seed)
tokens_init[:, optim_mask] = np.random.randint(100, tokenizer.vocab_size, tokens_init[:, optim_mask].shape)
def tokens_to_array(tokens):
    token_array = jnp.asarray(tokens)
    if len(token_array) >= bs_start:
        token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq")
    return token_array
def run_tokens(token_array, grad_metric=None):
    if not isinstance(token_array, pz.nx.NamedArray):
        token_array = tokens_to_array(token_array)
    inputs = llama.inputs.from_basic_segments(token_array)
    if grad_metric:
        @partial(jax.grad, has_aux=True)
        def lwg(x):
            logits, resids = get_resids_one_hot_call(dataclasses.replace(inputs, tokens=x))
            resids = {resid.tag: resid.value for resid in resids}
            metric = grad_metric(logits, resids, inputs)
            return metric, (logits, resids)
        vocab = llama.select().at_instances_of(pz.nn.EmbeddingLookup).get_sequence()[0].table.embeddings.value.named_shape["vocabulary"]
        one_hots = pz.nx.nmap(lambda x: jax.nn.one_hot(x, vocab))(inputs.tokens).tag("vocabulary")
        grad, (logits, resids) = lwg(one_hots)
    else:
        logits, resids = get_resids_call(inputs)
    losses = loss_fn(logits, inputs)
    if not grad_metric:
        resids = {resid.tag: resid.value for resid in resids}
    return_vals = logits, losses, resids
    if grad_metric:
        return_vals = return_vals + (grad,)
    return return_vals

mask = jax.device_put(jnp.asarray(optim_mask), jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("sp")))

@jax.jit
def temper(logits, key, elites, grads):
    key_choice, key_random = jax.random.split(key)
    index = jax.random.randint(key_choice, (), 0, len(logits) - 1)
    key_categorical, key_uniform, key_bernoulli, key_randint, key_use_grads, key_mutations = jax.random.split(key_random, 6)
    logit = logits[index]
    elite = elites[index]
    grads = grads[index]
    
    logit = jnp.roll(logit, 1, 0)
    logit = logit * jax.random.uniform(key_uniform, minval=0, maxval=max_inv_temp)
    use_grads = jax.random.bernoulli(key_use_grads, p=0.7).astype(jnp.int_)
    logit = jax.lax.switch(use_grads, ((lambda x: x), (lambda x: jnp.where(grads, x, -jnp.inf))), logit)
    to_change = jax.random.bernoulli(key_bernoulli, max(.5, expected_changes - 1) / mask.sum(), mask.shape)
    definite_indices = jax.random.randint(key_randint, mask.shape[:-1], 0, mask.shape[-1])
    definite_mask = jax.nn.one_hot(definite_indices, to_change.shape[-1], dtype=jnp.bool_)
    to_change = to_change | definite_mask
    changed = jnp.where(mask & to_change,
                        jax.random.categorical(key_categorical, logit),
                        elite)
    
    key_swap, key_mutations = jax.random.split(key_mutations)
    swap_indices = jax.random.randint(key_swap, (2,), 0, mask.sum())
    swap_from, swap_to = jnp.nonzero(mask, size=len(mask))[0][swap_indices]
    key_swap, key_mutations = jax.random.split(key_mutations)
    do_swap = jax.random.bernoulli(key_swap, p=0.25)
    swapped = changed.at[swap_from].set(changed[swap_to]).at[swap_to].set(changed[swap_from])
    changed = jax.lax.cond(do_swap, lambda x: swapped, lambda x: x, changed)
    
    # key_delete, key_mutations = jax.random.split(key_mutations)
    # delete_index = jax.random.randint(key_delete, (), 0, mask.sum())
    # indices = jnp.cumsum(mask)
    # indices_base = jnp.arange(len(changed))
    # deleted = jnp.where(mask, jnp.where(indices > delete_index, indices_base + 1, indices_base), changed)
    # key_delete, key_mutations = jax.random.split(key_mutations)
    # do_delete = jax.random.bernoulli(key_delete, p=0.1)
    # changed = jax.lax.cond(do_delete, lambda x: deleted, lambda x: x, changed)
    
    return changed

# @partial(jax.jit, static_argnames=("key", "candidates", "expected_changes"))
def algo_iteration(elites, vector, key="resid_pre.16", candidates=128, seed=13, expected_changes=3, max_inv_temp=4, topk=128):
    elites = elites.untag("solutions").tag("batch")
    logits, _, _, grads = run_tokens(elites, grad_metric=lambda _l, r, _i: (r[key][{"seq": -1}].untag("embedding") * vector).sum().data_array.mean())
    grads = pz.nx.nmap(lambda x: x >= jax.lax.top_k(x, topk)[0][-1])(grads.untag("vocabulary")).tag("vocabulary")
    logits = logits.untag("batch").tag("elites")

    tempered_samples = pz.nx.nmap(temper)(
        logits.untag("elites", "seq", "vocabulary"),
        pz.nx.wrap(jax.random.split(jax.random.key(seed), candidates), "batch"),
        elites.untag("batch", "seq"),
        grads.untag("batch", "seq", "vocabulary")).tag("seq")
    tempered_samples = sharding_util.name_to_name_device_put(tempered_samples, llama.mesh, dict(batch="dp", seq="sp"))
    _, new_losses, new_resids = run_tokens(tempered_samples)
    new_scores = (new_resids[key][{"seq": -1}].untag("embedding") * vector).sum().astype(new_losses.dtype)
    metrics = pz.nx.nmap(lambda *xs: jnp.stack(xs))(new_losses, new_scores).tag("metrics")
    solution_axes = [k for k in tempered_samples.named_shape.keys() if k != "seq"]
    solutions = tempered_samples.untag(*solution_axes).flatten().tag("solutions").unwrap("solutions", "seq")
    metrics = metrics.untag(*(k for k in solution_axes if k != "seq")).flatten().tag("solutions").unwrap("solutions", "metrics")
    return solutions, metrics


best_metrics = None
best = tokens_to_array(tokens_init).untag("batch").tag("solutions")
xent_min = 5
xent_max = 50
weights = jnp.stack((
    jnp.linspace(-xent_max, -xent_min, MAX_ELITES),
    jnp.ones(MAX_ELITES),
), -1)
@partial(jax.jit, donate_argnums=(0, 1))
def combine_solutions(best_metrics, best, metrics, solutions):
    if best_metrics is not None:
        best_metrics = jnp.concatenate((best_metrics, metrics), 0)
        best = pz.nx.nmap(lambda a, b: jnp.concatenate((a, b)))(
            best.untag("solutions"),
            pz.nx.wrap(solutions, "solutions", "seq").untag("solutions")
        ).tag("solutions").unwrap("solutions", "seq")
    else:
        best_metrics = metrics
        best = solutions
    elite_mask = (best_metrics[None, :] * weights[:, None]).sum(-1).argmax(1)
    best_metrics = best_metrics[elite_mask]
    best = pz.nx.wrap(best[elite_mask], "solutions", "seq")
    return best_metrics, best
for seed in (bar := trange(1_000)):
    solutions, metrics = algo_iteration(best, vector, seed=seed)
    best_metrics, best = combine_solutions(best_metrics, best, metrics, solutions)
    m = {}
    for index in range(MAX_ELITES):
        i = index
        m |= {f"decoded.{i}": tokenizer.decode(best[{"solutions": index}].unwrap("seq")),
              f"loss.{i}": best_metrics[index][0], f"score.{i}": best_metrics[index][1]}
    bar.set_postfix(**m)

  0%|          | 0/1000 [00:00<?, ?it/s]

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/jax/_src/numpy/array_methods.py:68: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return lax_numpy.astype(arr, dtype, copy=copy, device=device)
/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/jax/_src/numpy/array_methods.py:68: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return lax_numpy

KeyboardInterrupt: 

In [ ]:
_, resids = get_resids_call(inputs)